In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from getpass import getpass
import time
import urllib
import urllib.request
import requests
from io import BytesIO
import pandas as pd
from PIL import Image
import chromedriver_autoinstaller as chromedriver
from webdriver_manager.chrome import ChromeDriverManager
chromedriver.install()
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import xlsxwriter
import xlwings as xw
import openpyxl
from openpyxl import load_workbook,Workbook
from openpyxl.styles import colors
from openpyxl.styles import Font, Color
from openpyxl.styles.alignment import Alignment
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from xlwings import constants
from openpyxl.drawing.image import Image
import xlrd 
options = Options()
options.add_argument("start-maximized")
from IPython.display import display, HTML

In [20]:
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install())) 



INFO:WDM:

====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\aakas\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\aakas\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [21]:
url = "http://jntuhsit.org/"
driver.get(url)
driver.find_element(By.XPATH,"//a[@href='http://14.139.82.140']").click()
driver.switch_to.window(driver.window_handles[1])
time.sleep(2)

In [22]:
# a = ['21031F0002','21031F0005']
a = [f"21031F00{i:02d}" for i in range(1,31)]
# My_filename = 'MCA_1st_Sem_Results_Automated_New.xlsx'
My_filename = 'test.xlsx'
All_Results = pd.DataFrame(columns=['ROLL-NUMBER','NAME','SGPA'])
wbn = Workbook()
wbn.save(My_filename)
wbn.close()
for i in a:
    driver.find_element(By.ID,'uid').send_keys(i)
    if i=="21031F0005":
        driver.find_element(By.ID,'pwd').send_keys(getpass(f"{i} Enter Your Password : "))
    else:
        driver.find_element(By.ID,'pwd').send_keys("sit123")
    driver.find_element(By.CSS_SELECTOR,"input[type='submit']").click()
    if driver.find_element(By.XPATH,".//div[contains(@class,'alert')]").text == '×\n  Failed !  Invalid LoginID or Password....':
        continue
    else:
        time.sleep(1)
        driver.find_element(By.XPATH,"//div[@id='myNavBar']/ul/li[4]").click()
        time.sleep(1)
        driver.find_element(By.XPATH,"//a[@href='viewResults']").click()
        time.sleep(1)
        if driver.find_element(By.XPATH,".//div[contains(@class,'alert')]").text == '×\n  Failed !  Results records not found.':
            driver.find_element(By.XPATH,"//a[@href='logout']").click()
            continue
        else:
            driver.find_element(By.XPATH,"//select[@id='ecode']/option[text()='M.C.A. I Year I Semester (R20) (Reg) - April, 2022']").click()
            time.sleep(1)
            driver.find_element(By.XPATH,".//button[contains(@class,'btn')]").click()
            time.sleep(1)
            BS   = BeautifulSoup(driver.page_source,'html.parser')
            df1  = pd.DataFrame(data=driver.find_element(By.XPATH,"//*[@id='resBlock']/table/tbody/tr[1]").text.split('\n')+[driver.find_element(By.XPATH,"//*[@id='resBlock']/table/tbody/tr[2]").text])
            df2  = pd.DataFrame(data=[driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[{i}]").text for i in range(3,6)])
            df3  = pd.DataFrame(data=[[j] + [ int(driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[6]/td/table/tbody/tr[{j}]/td[{i}]").text) if i==5 or i==6 else  driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[6]/td/table/tbody/tr[{j}]/td[{i}]").text for i in range(2,8) ] for j in range(1,10)],columns=[driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[6]/td/table/thead/tr/th[{i}]").text  for i in range(1,8)])
            All_Results.loc[len(All_Results)] = [i,df2[0][1].split(":")[1].strip(' '),round(sum(df3['Grade Point'] * df3['Credits'])/sum(df3['Credits']),2)]
            wb = load_workbook(My_filename)
            wb.create_sheet(title=i)
            ws = wb[i]
            df1_rows = dataframe_to_rows(df1, index=False,header=False)
            df2_rows = dataframe_to_rows(df2, index=False,header=False)
            df3_rows = dataframe_to_rows(df3, index=False)
            for r_idx, row in enumerate(df1_rows, 2):
                for c_idx, value in enumerate(row, 6):
                    ws.cell(row=r_idx, column=c_idx, value=value)
            for r_idx, row in enumerate(df2_rows, 10):
                for c_idx, value in enumerate(row, 6):
                    ws.cell(row=r_idx, column=c_idx, value=value)
            for r_idx, row in enumerate(df3_rows, 15):
                for c_idx, value in enumerate(row, 4):
                    ws.cell(row=r_idx, column=c_idx, value=value)
            for (igty,jfg) in zip(['E','F','H'],[14,65,12]):
                        ws.column_dimensions[igty].width = jfg
            for per in [int(ifrt) for ifrt in range(1,11)]:
                    for jzdd in [int(eri) for eri in range(1,31)]:
                        ws.cell(jzdd,per).alignment = Alignment(horizontal='center')    
            for c in [4,5,6,7,8,9,10]:
                ws.cell(15,c).font = Font(bold=True)
            for a,b in zip([2,3,4,5,6],['f7118b','0a21a8','0a21a8','0a21a8','ff0000']):
                ws.cell(a,6).font = Font(color=b,bold=True)
            for (sft,rtf,rty) in zip([25,25,27,27],[8,9,6,7],["=SUMPRODUCT((H16:H24)*(I16:I24))","=SUM((I16:I24))","SGPA : ","=ROUND((H25/I25),3)"]):
                ws.cell(sft,rtf).value = rty
                ws.cell(sft,rtf).font = Font(bold=True)
            img = Image('Jntuh Logo.png')
            img.height = 150
            img.width = 170
            ws.add_image(img,'D3')  
            img = Image(BytesIO(urllib.request.urlopen(BS.find('table',class_='borderless').find('img',alt='PHOTO')['src']).read()))
            img.height = 110
            img.width = 110
            ws.add_image(img,'G8')
            ws.column_dimensions['G'].width = 15
            ws.cell(15,7).alignment = Alignment(horizontal='center')
            wb.save(My_filename)
    driver.find_element(By.XPATH,"//a[@href='logout']").click()
    print(i,":",driver.find_element(By.CLASS_NAME,'alert-success').text.strip(".")[2:40])
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()
wb = load_workbook(My_filename)
wb.create_sheet(title='All_Results')
ws = wb['All_Results']
All_Results_df = dataframe_to_rows(All_Results, index=False)
for ridx, rw in enumerate(All_Results_df, 1):
    for cidx, value in enumerate(rw,1):
        ws.cell(row=ridx, column=cidx, value=value)
for (irty,jstdr) in zip(['A','B','C'],[18,50,15]):
            ws.column_dimensions[irty].width = jstdr
for fnt in [1,2,3]:
    ws.cell(1,fnt).font = Font(bold=True)
for psrg in [int(idfs) for idfs in range(1,11)]:
    for jree in [int(ier) for ier in range(1,31)]:
        ws.cell(jree,psrg).alignment = Alignment(horizontal='center')        
if 'Sheet' in wb.sheetnames:
    wb.remove(wb['Sheet'])
wb.save(My_filename)

21031F0002 :   Success !  Successfully Logged out
21031F0004 :   Success !  Successfully Logged out
21031F0005 Enter Your Password : ········
21031F0005 :   Success !  Successfully Logged out
21031F0007 :   Success !  Successfully Logged out
21031F0008 :   Success !  Successfully Logged out
21031F0009 :   Success !  Successfully Logged out
21031F0010 :   Success !  Successfully Logged out
21031F0012 :   Success !  Successfully Logged out
21031F0014 :   Success !  Successfully Logged out
21031F0015 :   Success !  Successfully Logged out
21031F0018 :   Success !  Successfully Logged out
21031F0021 :   Success !  Successfully Logged out
21031F0022 :   Success !  Successfully Logged out
21031F0024 :   Success !  Successfully Logged out
21031F0025 :   Success !  Successfully Logged out
21031F0029 :   Success !  Successfully Logged out


In [11]:
len(All_Results)

2

In [ ]:
# # a = ['21031F0005']
# a = [f"21031F00{i:02d}" for i in range(1,31)]
# All = {}
# wbn = Workbook()
# wbn.save('MCA_1st_Sem_Results_Automated.xlsx')
# wbn.close()
# for i in a:
#     driver.find_element(By.ID,'uid').send_keys(i)
#     if i=="21031F0005":
#         driver.find_element(By.ID,'pwd').send_keys(getpass(f"{i} Enter Your Password : "))
#     else:
#         driver.find_element(By.ID,'pwd').send_keys("sit123")
#     driver.find_element(By.CSS_SELECTOR,"input[type='submit']").click()
#     if driver.find_element(By.XPATH,".//div[contains(@class,'alert')]").text == '×\n  Failed !  Invalid LoginID or Password....':
#         continue
#     else:
#         time.sleep(1)
#         driver.find_element(By.XPATH,"//div[@id='myNavBar']/ul/li[4]").click()
#         time.sleep(1)
#         driver.find_element(By.XPATH,"//a[@href='viewResults']").click()
#         time.sleep(1)
#         if driver.find_element(By.XPATH,".//div[contains(@class,'alert')]").text == '×\n  Failed !  Results records not found.':
#             driver.find_element(By.XPATH,"//a[@href='logout']").click()
#             continue
#         else:
#             driver.find_element(By.XPATH,"//select[@id='ecode']/option[text()='M.C.A. I Year I Semester (R20) (Reg) - April, 2022']").click()
#             time.sleep(1)
#             driver.find_element(By.XPATH,".//button[contains(@class,'btn')]").click()
#             time.sleep(1)
#             BS = BeautifulSoup(driver.page_source,'html.parser')
#             df1  = pd.DataFrame(data=driver.find_element(By.XPATH,"//*[@id='resBlock']/table/tbody/tr[1]").text.split('\n')+[driver.find_element(By.XPATH,"//*[@id='resBlock']/table/tbody/tr[2]").text])
#             df2  = pd.DataFrame(data=[driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[{i}]").text for i in range(3,6)])
#             df3  = pd.DataFrame(data=[[j] + [ int(driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[6]/td/table/tbody/tr[{j}]/td[{i}]").text) if i==5 or i==6 else  driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[6]/td/table/tbody/tr[{j}]/td[{i}]").text for i in range(2,8) ] for j in range(1,10)],columns=[driver.find_element(By.XPATH,f"//*[@id='resBlock']/table/tbody/tr[6]/td/table/thead/tr/th[{i}]").text  for i in range(1,8)])
#             wb = load_workbook('MCA_1st_Sem_Results_Automated.xlsx')
#             wb.create_sheet(title=i)
#             ws = wb[i]
#             df1_rows = dataframe_to_rows(df1, index=False,header=False)
#             df2_rows = dataframe_to_rows(df2, index=False,header=False)
#             df3_rows = dataframe_to_rows(df3, index=False)
#             for r_idx, row in enumerate(df1_rows, 2):
#                 for c_idx, value in enumerate(row, 6):
#                     ws.cell(row=r_idx, column=c_idx, value=value)
#             for r_idx, row in enumerate(df2_rows, 10):
#                 for c_idx, value in enumerate(row, 6):
#                     ws.cell(row=r_idx, column=c_idx, value=value)
#             for r_idx, row in enumerate(df3_rows, 15):
#                 for c_idx, value in enumerate(row, 4):
#                     ws.cell(row=r_idx, column=c_idx, value=value)
#             for (i,j) in zip(['E','F','H'],[14,65,12]):
#                         ws.column_dimensions[i].width = j
#             for p in [int(i) for i in range(1,11)]:
#                     for j in [int(i) for i in range(1,31)]:
#                         ws.cell(j,p).alignment = Alignment(horizontal='center')    
#             for c in [4,5,6,7,8,9,10]:
#                 ws.cell(15,c).font = Font(bold=True)
#             for a,b in zip([2,3,4,5,6],['f7118b','0a21a8','0a21a8','0a21a8','ff0000']):
#                 ws.cell(a,6).font = Font(color=b,bold=True)
#             for (sft,rtf,rty) in zip([25,25,27,27],[8,9,6,7],["=SUMPRODUCT((H16:H24)*(I16:I24))","=SUM((I16:I24))","SGPA : ","=ROUND((H25/I25),3)"]):
#                 ws.cell(sft,rtf).value = rty
#                 ws.cell(sft,rtf).font = Font(bold=True)
#             img = Image('Jntuh Logo.png')
#             img.height = 150
#             img.width = 170
#             ws.add_image(img,'D3')  
#             img = Image(BytesIO(urllib.request.urlopen(BS.find('table',class_='borderless').find('img',alt='PHOTO')['src']).read()))
#             img.height = 110
#             img.width = 110
#             ws.add_image(img,'G8')
#             ws.column_dimensions['G'].width = 15
#             ws.cell(15,7).alignment = Alignment(horizontal='center')
#             wb.save('MCA_1st_Sem_Results_Automated.xlsx')
#     driver.find_element(By.XPATH,"//a[@href='logout']").click()
#     print(i,":",driver.find_element(By.CLASS_NAME,'alert-success').text.strip(".")[2:40])
# driver.close()
# driver.switch_to.window(driver.window_handles[0])
# driver.close()